In [ ]:
#enabling tensorflow with TPU support
%tensorflow_version 1.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

TensorFlow 1.x selected.
Tensorflow version 1.15.2
Running on TPU  ['10.23.164.122:8470']
INFO:tensorflow:Initializing the TPU system: 10.23.164.122:8470
INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Querying Tensorflow master (grpc://10.23.164.122:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 4347880099433207690)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 5118217055574266196)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 10838393059695317128)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 1417104410370385611

In [ ]:
#Importing necessary modules
print(tf.__version__)
import tensorflow_hub as hub
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity

module_url = "https://tfhub.dev/google/universal-sentence-encoder/1?tf-hub-format=compressed"

# Import the Universal Sentence Encoder's TF Hub module
embed = hub.Module(module_url)

1.15.2


In [ ]:
#Function to find semantic similarity of two sentences using Google's Universal Sentence Encoder
def semantic_similarity(cap1,cap2):
    messages=[cap1,cap2]
    similarity_input_placeholder = tf.placeholder(tf.string, shape=(None))
    similarity_message_encodings = embed(similarity_input_placeholder)
    with tf.Session() as session:
        session.run(tf.global_variables_initializer())
        session.run(tf.tables_initializer())
        message_embeddings_ = session.run(similarity_message_encodings, feed_dict={similarity_input_placeholder: messages})    
    return cosine_similarity(message_embeddings_[0].reshape(1,-1),message_embeddings_[1].reshape(1,-1))

In [ ]:
#Installing necessary modules
!pip install youtube-dl
!pip install download-youtube-subtitle
!pip install pafy
!pip install ffmpeg-python
!pip3 install --upgrade speechrecognition

     |████████████████████████████████| 1.8MB 3.5MB/s 
     |████████████████████████████████| 81kB 2.6MB/s 
  Created wheel for download-youtube-subtitle: filename=download_youtube_subtitle-1.0.1-cp36-none-any.whl size=8796 sha256=747de47d09b9cdf109e0d54f11c4d67289c7d09f82449cec54ccf398498e9d41
  Stored in directory: /root/.cache/pip/wheels/1c/90/d3/77d2a80c78c00533592d4c8200d4d213e1601aa1b396e85ad8
  Created wheel for fire: filename=fire-0.3.1-py2.py3-none-any.whl size=111005 sha256=d6004327a0af28c90800f06c67d84f54c8b18de32c352d1f6ad2426e50d88aaa
  Stored in directory: /root/.cache/pip/wheels/c1/61/df/768b03527bf006b546dce284eb4249b185669e65afc5fbb2ac
Successfully built download-youtube-subtitle fire


In [ ]:
input =  '_PwhiWxHK8o'#@param {type:"string"}

In [ ]:
#Generating the subtitle file for the given ID
import os
os.system('dl-youtube-cc '+input+' --translation False --to_json=True --output_file="input.json"')

0

In [ ]:
#Reading the subtitle file generated
import json
f=open(r"/content/input.json")
data=json.load(f)["original"]

In [ ]:
#Filtering
transcript=[]
for i in range(len(data)-1):
  if(data[i]['text']):
    transcript.append([data[i]['text'].replace('\n',' '),[float(data[i]['start']),float(data[i+1]['start'])]])

In [ ]:
len(transcript)

868

In [ ]:
#Finding the length of the input Video
import pafy
url = "https://youtu.be/"+input
video = pafy.new(url)
length=video.length

In [ ]:
#Getting the last transcript
if(data[-1]["text"]):
  transcript.append([data[-1]['text'].replace('\n',' '),[float(data[-1]['start']),length]])

In [ ]:
transcript

[['PATRICK WINSTON: So where are we?', [9.234, 10.3]],
 ['We started off with simple methods for learning stuff.', [10.3, 14.962]],
 ['Then, we talked a little about a purchase of learning that',
  [14.962, 20.73]],
 ["we're vaguely inspired by.", [20.73, 24.556]],
 ['The fact that our heads are stuffed with neurons, and that',
  [24.556, 27.3]],
 ['we seemed to have evolved from primates.', [27.3, 31.095]],
 ['Then, we talked about looking at the problem and address the',
  [31.095, 34.94]],
 ['issue of [? phrenology ?]', [34.94, 36.41]],
 ["and how it's possible to learn concepts.", [36.41, 40.43]],
 ["But now, we're coming full circle back to the beginning", [40.43, 43.7]],
 ['and thinking about how to divide up a space with', [43.7, 47.99]],
 ['decision boundaries.', [47.99, 49.93]],
 ['But whereas, you do it with a neural net or a nearest', [49.93, 54.58]],
 ['neighbors or a ID tree.', [54.58, 56.51]],
 ['Those are very simple ideas that work very often.', [56.51, 62.115]],
 ["Tod

In [ ]:
#Speech Recognition Code to enable microphone

from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };            
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {            
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data); 
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});
      
</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  
  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)
  
  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr

In [ ]:
#Speech to Text conversion to obtain the voice query as text

text=0
import speech_recognition
r = speech_recognition.Recognizer()
print("Speak Anything :")
audio, sr = get_audio()
#print(type(audio))
import numpy as np
from scipy.io.wavfile import write


write('test.wav', 44100, audio)
with speech_recognition.AudioFile('test.wav') as source:
        audio = r.record(source)  # read the entire audio file                  
        try:
            text = r.recognize_google(audio)
            print("You said : {}".format(text))
        except:
            print("Sorry could not recognize what you said")

Speak Anything :


You said : contain


In [ ]:
#Mounting Drive

from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
text="constraint"

In [ ]:
#Processing the transcripts to generate the final video segments according to the voice query given by the user
#Cutting threshold - 2(represented by variable c)

final=[]
from IPython.display import HTML
from base64 import b64encode
c=0
i= 0
#name="videoplayback.mp4"
while(i<len(transcript)):
  print(transcript[i][0])
  f=open("drive/My Drive/Mini-Project/audio1.txt",'w')
  f.write('i = '+str(i)+"\nfinal = "+str(final)+"\n")
  f.close()
  if(text in transcript[i][0] or semantic_similarity(transcript[i][0],text)>=0.6):
    c=0
    j=i+1
    k=i
    start=transcript[i][1][0]
    end=transcript[i][1][1]
    while(j<len(transcript)):
      if(semantic_similarity(transcript[j][0],transcript[k][0])>=0.5):
        k=j
        c=0
        end=transcript[j][1][1]
      else:
        c+=1
        if(c>=2):
          final.append([transcript[i][0],[start,end]])
          i=j
          break
      j+=1
    print("match")
    print(final)
  i+=1

another space with a better perspective.
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Now, the history lesson, all this stuff feels fairly new.
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


It feels like it's younger than you are.
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


becomes famous for something that everybody knows about
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


today who does machine learning.
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [ ]:
#Final List generated

final = [["So we don't have enough constraint here to fix a", [462.74, 465.75]], ["additional constraints whether you're toward putting enough", [485.79, 493.33]], ['to be equal to or greater than 1.', [530.05, 531.37]], ['that this decision function gives the', [546.12, 548.55]], ["going to add the additional constraint that it's going to", [831.14, 833.54]], ['some constraints.', [874.34, 876.416]], ['equation that constrains the samples', [1045.954, 1055.61]], ['Well, I decided I was going to enforce this constraint.', [1134.21, 1139.27]], ['Then, I used the constraint to plug back some values here.', [1146.105, 1149.4]], ['minimum of, the extremum of.', [1355.68, 1358.236]], ["a function with constraints, then we're going to have to", [1408.462, 1415.922]], ['the constraints anymore.', [1423.35, 1425.09]], ['constraints.', [1456.23, 1457.48]], ['And then, we write down the constraint.', [1463.412, 1467.575]], ['So the constraint is y sub i times vector, w, dotted with', [1473.83, 1489.03]], ["It's back to that constraint up there, where each", [1514.425, 1516.3]], ['Finding the maximum--', [1558.33, 1564.76]], ["Let's take the partial of L, the Lagrangian, with respect", [1572.89, 1604.88]], ['This is a so-called quadratic optimization problem.', [1849.62, 1854.772]], ['linear sum of the samples.', [1878.77, 1881.265]], ['sums together into a double summation, so I have to keep', [1965.28, 1969.99]], ['as the sum of the alphas.', [2058.32, 2060.059]], ['this Lagrangian as--', [2111.14, 2115.56]], ['expression.', [2185.44, 2186.95]], ['optimization.', [2313.85, 2315.1]], ['optimization.', [2360.63, 2362.406]], ['into the solution, the weights on the Lagrangian multipliers', [2370.97, 2380.03]], ['gives examples that are not linearly separable, then we', [2491.85, 2497.68]], ['product of those two vectors in another space.', [2644.65, 2651.2]]]

In [ ]:
#Sorting the final video segments according to start time, if required
final=sorted(final,key=lambda x:x[1][0])

In [ ]:
final

[["So we don't have enough constraint here to fix a", [462.74, 465.75]],
 ["additional constraints whether you're toward putting enough",
  [485.79, 493.33]],
 ['to be equal to or greater than 1.', [530.05, 531.37]],
 ['that this decision function gives the', [546.12, 548.55]],
 ["going to add the additional constraint that it's going to",
  [831.14, 833.54]],
 ['some constraints.', [874.34, 876.416]],
 ['equation that constrains the samples', [1045.954, 1055.61]],
 ['Well, I decided I was going to enforce this constraint.',
  [1134.21, 1139.27]],
 ['Then, I used the constraint to plug back some values here.',
  [1146.105, 1149.4]],
 ['minimum of, the extremum of.', [1355.68, 1358.236]],
 ["a function with constraints, then we're going to have to",
  [1408.462, 1415.922]],
 ['the constraints anymore.', [1423.35, 1425.09]],
 ['constraints.', [1456.23, 1457.48]],
 ['And then, we write down the constraint.', [1463.412, 1467.575]],
 ['So the constraint is y sub i times vector, w, dotted wi

In [ ]:
#Merging the generated list if two consequtive videos are supposed to be continuous based on pre-decided threshold

length=len(final)
i=0
while (i<(length-1)):
  if(final[i+1][1][0]-final[i][1][1]<=60):
    final[i][1][1]=final[i+1][1][1]
    final.pop(i+1)
    length-=1
  else:
    i+=1

In [ ]:
final

[["So we don't have enough constraint here to fix a", [462.74, 548.55]],
 ["going to add the additional constraint that it's going to",
  [831.14, 876.416]],
 ['equation that constrains the samples', [1045.954, 1055.61]],
 ['Well, I decided I was going to enforce this constraint.',
  [1134.21, 1149.4]],
 ['minimum of, the extremum of.', [1355.68, 1604.88]],
 ['This is a so-called quadratic optimization problem.', [1849.62, 1881.265]],
 ['sums together into a double summation, so I have to keep',
  [1965.28, 1969.99]],
 ['as the sum of the alphas.', [2058.32, 2115.56]],
 ['expression.', [2185.44, 2186.95]],
 ['optimization.', [2313.85, 2380.03]],
 ['gives examples that are not linearly separable, then we',
  [2491.85, 2497.68]],
 ['product of those two vectors in another space.', [2644.65, 2651.2]]]

In [ ]:
#Filtering based on duration

final=list(filter(lambda x:x[1][1]-x[1][0]>=7,final))

In [1]:
final=[["So we don't have enough constraint here to fix a", [462.74, 548.55]],
 ["going to add the additional constraint that it's going to",
  [831.14, 876.416]],
 ['equation that constrains the samples', [1045.954, 1055.61]],
 ['Well, I decided I was going to enforce this constraint.',
  [1134.21, 1149.4]],
 ['minimum of, the extremum of.', [1355.68, 1604.88]],
 ['This is a so-called quadratic optimization problem.', [1849.62, 1881.265]],
 ['as the sum of the alphas.', [2058.32, 2115.56]],
 ['optimization.', [2313.85, 2380.03]]]

In [4]:
#Playing of results

import ipywidgets as widgets
from IPython.display import display,clear_output
from IPython.display import YouTubeVideo,HTML
button = widgets.Button(description="Play Next")
output = widgets.Output()
x=0
def on_button_clicked(b):
  # Display the message within the output widget.
  with output:
    clear_output()
    global x
    s="https://www.youtube.com/embed/_PwhiWxHK8o?start="+str(int(final[x][1][0]))+"&end="+str(int(final[x][1][1]))
    #display(YouTubeVideo(s))
    display(HTML('<iframe width="560" height="315" src='+s+' frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>'))
    print("Result ",x+1)
    x+=1
    x%=len(final)

button.on_click(on_button_clicked)
display(button, output)

Button(description='Play Next', style=ButtonStyle())

Output()